In [27]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import time

def image_data_generator(data_folder, img_size=(224, 224), batch_size=32, color_mode='rgb'):
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    generator = datagen.flow_from_directory(
        data_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        color_mode=color_mode
    )
    return generator

In [28]:
def ResNet50_model(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
    return model

In [29]:
base_path = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold'
train_data_folder = os.path.join(base_path, 'Train_Image')
val_data_folder = os.path.join(base_path, 'Validation_Image')
test_data_folder = os.path.join(base_path, 'Test_Image')

# 데이터 제너레이터 생성
train_generator = image_data_generator(train_data_folder, batch_size=128)
val_generator = image_data_generator(val_data_folder, batch_size=128)
test_generator = image_data_generator(test_data_folder, batch_size=128)

# 모델 정의
# Input shape은 (None, None, None)으로 설정하여 모든 크기의 이미지를 받을 수 있게 합니다.
# 실제 이미지 크기는 제너레이터가 알아서 조절합니다.
model = ResNet50_model(input_shape=(224, 224, 3))

# 배치 크기와 에포크 설정
epochs = 10

# 모델 훈련
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

# 테스트하기 전에 시간 측정 시작
start_time = time.time()

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)

# 시간 측정 종료 및 결과 출력
end_time = time.time()
elapsed_time = end_time - start_time

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)
print(f"Test Duration: {elapsed_time:.4f} seconds")

Found 320000 images belonging to 2 classes.
Found 40000 images belonging to 2 classes.
Found 37500 images belonging to 2 classes.
Epoch 1/10
2500/2500 [==============================] - 478s 190ms/step - loss: 0.6954 - accuracy: 0.5336 - auc: 0.5468 - val_loss: 0.6875 - val_accuracy: 0.5415 - val_auc: 0.5728
Epoch 2/10
2500/2500 [==============================] - 475s 190ms/step - loss: 0.6868 - accuracy: 0.5417 - auc: 0.5603 - val_loss: 0.6850 - val_accuracy: 0.5500 - val_auc: 0.5725
Epoch 3/10
2500/2500 [==============================] - 476s 191ms/step - loss: 0.6859 - accuracy: 0.5442 - auc: 0.5633 - val_loss: 0.6846 - val_accuracy: 0.5467 - val_auc: 0.5749
Epoch 4/10
2500/2500 [==============================] - 476s 191ms/step - loss: 0.6838 - accuracy: 0.5481 - auc: 0.5700 - val_loss: 0.6819 - val_accuracy: 0.5533 - val_auc: 0.5819
Epoch 5/10
2500/2500 [==============================] - 479s 191ms/step - loss: 0.6822 - accuracy: 0.5533 - auc: 0.5753 - val_loss: 0.6832 - val_accur

In [30]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(test_generator, steps=len(test_generator))

# 예측된 레이블을 이진 레이블로 변환합니다.
# 이진 분류에서는 0.5를 기준으로 설정합니다.
predicted_classes = (predictions > 0.5).astype(int)

# 실제 레이블을 얻습니다.
true_classes = test_generator.classes

# 혼동 행렬 생성
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# 혼동 행렬 출력
print("Confusion Matrix:")
print(conf_matrix)

293/293 [==============================] - 49s 166ms/step
Confusion Matrix:
[[13423  6577]
 [11622  5878]]
